# Evaluator+PrikazTop5Guess

### Ovaj program je demonstracija Evaluatora, koristi se mreza koja je istrenirana na Logima iz poznatih 25 klasa.
### Programu treba directory gdje se nalaze slike iz test skupa. Istrenirani Model. Tresholdi su pronadjeni preko Gridsearcha. Program ispisuje prvih 5 najbolji guesseva mreze, na nacin da ih poreda po broju proposala (i po prosjecnim vjerojatnostima)

In [1]:
from __future__ import division, print_function, absolute_import
import tflearn 
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

from tqdm import tqdm
import os
import json
import numpy as np
from random import shuffle
import cv2 
import shutil
import csv
import time
import random


TreshMax=90
topk=5
MODELNAME = "punocisci29-06_mriza_best9862"

#direktoriji koji su bitni za rad programa
SOURCEIMG = "C:/Users/User/Desktop/KODOVI_MOZGALO/MacLearnImg/ResIMG/"

LR = 0.001
IMGSIZE1 = 60
IMGSIZE2 = 40

#Get labels for our logos 
def getLabels():
    with open('imgLabels.json') as f:
        return json.load(f)

#Loada keyeve
def getMyKeys():
    with open('claskey.json') as f:
        return json.load(f)

#loada omjere za klase tj koliko je puta selective search predložio groundtruth/ukupni broj prijedloga
def getClassOmjer():
    with open('omjerclass.json') as f:
        return json.load(f)

#proces data from given picture into
def processTestData(img):
    testData = []
    
    for props in os.listdir(SAVELOC + img+ "/"):
        #print(cv2.imread(os.path.join(SAVELOC, props)))
        cvimg = cv2.imread(os.path.join(SAVELOC + img+ "/", props), 0)
        imgNum = props.split('.')[0]
        #print (np.array(cvimg))
        testData.append([np.array(cvimg), imgNum])
    return testData


if __name__ == "__main__":

    labels = getLabels()   
    myKeys = getMyKeys()   
    tempRes = {}
    #ovdje slazemo navedeni dictionary
    for key in myKeys.keys():
        tempRes.update({myKeys[key]: 0})

    #malo optimizacije
    cv2.setUseOptimized(True)
    cv2.setNumThreads(8)
    
    #preloadanje training seta iz fiknsog filea. sadrži malo manje od pola milijona slika
    print("\n\n")
    print("Starting with convnet. Fingers crossed\n")
    #network = tflearn.input_data(shape=[None, IMGSIZE1, IMGSIZE2, 1], data_augmentation=img_aug, name ='input')
    
    network = input_data(shape=[None, IMGSIZE1, IMGSIZE2, 1], name='input')
    network = conv_2d(network, 50, 5, activation='relu')
    network = max_pool_2d(network, 2)
    #Normalization purposed to speed up training process
    network = local_response_normalization(network)
    network = conv_2d(network, 50, 5, activation='relu')
    network = avg_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = conv_2d(network, 100, 5, activation='relu')
    network = avg_pool_2d(network, 2)
    #network = local_response_normalization(network)
    
    network = fully_connected(network, 1000, activation='relu')
    network = fully_connected(network, 1000, activation='relu')
    network = fully_connected(network, 200, activation='relu')
    network = fully_connected(network, 25, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR,
                        loss='categorical_crossentropy', name='targets')

    
    model = tflearn.DNN(network, tensorboard_dir='log' + MODELNAME)

    if os.path.exists("{}.meta".format(MODELNAME)):
        model.load(MODELNAME)
        print("Model Loaded")
        
    #testne slike bi trebale biti spremljene u odgovarajuci SOURCEIMG kako je navedeno gore u originalnom formatu.
    testImages =  [img for img in os.listdir(SOURCEIMG)]
    #u results spremamo zavrne podatke. broj slike : klasa kojoj je model odlucio da pripada.
    results = {}
    
    Height=400
    Width=220
    lista=[]
    temporaryResults = tempRes
    indexslike=0
    #prolazim po slikama
    for img in tqdm(testImages): 
        
        index = img.split('.')[0]
        
        intindex=int(index)
        
        for key in myKeys.keys(): 
            temporaryResults[myKeys[key]]=0
        
        try:
            im = cv2.imread(SOURCEIMG +img)
            im = cv2.resize(im, (Width, Height))
            ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
            ss.setBaseImage(im)
    
            #ss.switchToSingleStrategy()
            ss.switchToSelectiveSearchFast()
            # run selective search segmentation on input image
                
            #funkcija vrati array sa koordinatama rectangleova [x1 y1 w h] w->sirina h->visina dakle x1 y1 x1+w y1+h so koord od recta
            rects = ss.process()
            threshold_rects=[]
    
            #cuva samo odredjene rectangle
            for i,rect in enumerate(rects):
                x, y, w, h = rect
                
                if (w>(2*h/3) and w*h<Height*Width/15 and w*h>Height*Width/250): 
                #if (w*h >= Height*Width/100):
                    threshold_rects.append(rect)
            
            n_proposala=int(len(threshold_rects))
            max_ind=np.zeros(shape=(n_proposala, 2))
            sume = np.zeros(shape=(25, 2))
            average = np.zeros(shape=(25, 1))
            imOut = im.copy()
            for i,rect in enumerate(threshold_rects):
                x ,y, w, h = rect
           
                img_prop = imOut[y:y+h, x:x+w]
                proposal = cv2.resize(img_prop,(IMGSIZE1, IMGSIZE2))
                proposal = cv2.cvtColor(proposal, cv2.COLOR_BGR2GRAY)
    
                img_data=np.array(proposal)
                orig = img_data
                data = img_data.reshape(-1, IMGSIZE1, IMGSIZE2, 1)
                model_out = model.predict(data)[0]
                maximum=max(model_out)
                myInd = str(np.where(model_out==maximum)[0][0])
                max_ind[i][0]=maximum
                max_ind[i][1]=myInd
            
                        
            for i in range(0, n_proposala):
                sume[int(max_ind[i][1])][0] = max_ind[i][0]+sume[int(max_ind[i][1])][0]
                sume[int(max_ind[i][1])][1] = sume[int(max_ind[i][1])][1]+ 1
            for i in range(0, 25):
                if(sume[i][1] > 1):
                   average[i][0]=np.divide(sume[i][0], sume[i][1])   
                   
            ind = np.argpartition(average[:,0], -topk)[-topk:]
            ind = ind[average[ind,0].argsort()]
            #ind[np.argsort(average[ind][0])]
            #print(ind[0])
            
            print("\n")
            for n in range(0,topk):
                print("slika " + str(intindex) + " klasa " + str(myKeys[str(ind[n])]) + " P-> " + str(average[ind[n]][0]) + " regioni -> " + str(sume[ind[n]][1]) + "\n")
            
            maxi=0
            maxTotKoef=0
            for i in range (0,topk):
                if sume[ind[i]][1]<=2:
                        Pkoef=0
                else:
                    if average[ind[i]][0]>=0.99:
                        Pkoef=(average[ind[i]][0]*1000)-900
                    elif average[ind[i]][0]>=0.90:
                        Pkoef=(average[ind[i]][0]*100)-10
                    elif average[ind[i]][0]>=0.70:
                        Pkoef=(average[ind[i]][0]*100)-20
                    else:
                        Pkoef=0
                
                TotKoef=Pkoef+sume[ind[i]][1]
                
                if TotKoef>maxTotKoef:
                    maxTotKoef=TotKoef
                    maxi=i
      
            if maxTotKoef>=TreshMax:
                results.update({intindex : myKeys[str(ind[maxi])]})
            else:
                results.update({intindex : "Other"})  
                 
        except:
            print("U exceptu\n")
            results.update({intindex: "Other"})
            indexslike = indexslike +1
    '''
    with open('resultsAverage.csv', 'w',newline='') as csv_file:
        writer = csv.writer(csv_file)    
        for key in sorted(results):
            writer.writerow([results[key]])
    '''
    for key in sorted(results):
        print ("%s: %s" % (key, results[key]))

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)



Starting with convnet. Fingers crossed

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from C:\Users\User\Desktop\KODOVI_MOZGALO\FRANOV_EVALUATOR\Dorian final\DobarModel25klasa\punocisci29-06_mriza_best9862
Model Loaded


  0%|          | 0/36 [00:00<?, ?it/s]



slika 0 klasa Albertsons P-> 0.8167120433029007 regioni -> 17.0

slika 0 klasa BJs P-> 0.9365114830434322 regioni -> 16.0

slika 0 klasa Wegmans P-> 0.9495315253734589 regioni -> 2.0

slika 0 klasa HEB P-> 0.9593714475631714 regioni -> 2.0

slika 0 klasa Smiths P-> 0.9997538994340336 regioni -> 17.0



  3%|▎         | 1/36 [00:02<01:44,  2.98s/it]

...



slika 8 klasa CVSPharmacy P-> 0.6658904068171978 regioni -> 4.0

slika 8 klasa Albertsons P-> 0.7113544394572576 regioni -> 3.0

slika 8 klasa Walmart P-> 0.8084234800189734 regioni -> 8.0

slika 8 klasa WholeFoodsMarket P-> 0.8556009382009506 regioni -> 4.0

slika 8 klasa Safeway P-> 0.9559957012534142 regioni -> 16.0



 97%|█████████▋| 35/36 [01:36<00:02,  2.77s/it]



slika 9 klasa Wegmans P-> 0.4893400967121124 regioni -> 2.0

slika 9 klasa Albertsons P-> 0.554072351505359 regioni -> 12.0

slika 9 klasa Publix P-> 0.6844040509313345 regioni -> 8.0

slika 9 klasa StopShop P-> 0.711525841483048 regioni -> 7.0

slika 9 klasa Smiths P-> 0.9766857160462273 regioni -> 18.0



100%|██████████| 36/36 [01:38<00:00,  2.74s/it]


0: Smiths
1: Target
2: WholeFoodsMarket
3: Smiths
4: ShopRite
5: Walgreens
6: Other
7: SamsClub
8: Safeway
9: Smiths
10: SamsClub
11: Other
12: Other
13: WholeFoodsMarket
14: KingSoopers
15: Other
16: KingSoopers
17: Other
18: Other
19: HEB
20: Other
21: Other
22: Other
23: Other
24: Other
25: SamsClub
26: CVSPharmacy
27: BJs
28: SamsClub
29: Target
30: Other
31: Meijer
32: Other
33: Albertsons
34: Other
35: ShopRite
